In [1]:
import numpy as np
import pandas as pd
from functools import reduce

from sksurv.metrics import (
    concordance_index_censored,
    concordance_index_ipcw,
    cumulative_dynamic_auc,
    integrated_brier_score,
    brier_score,
    as_concordance_index_ipcw_scorer,
    as_cumulative_dynamic_auc_scorer,
    as_integrated_brier_score_scorer,
      
)

from sklearn.model_selection import KFold,GridSearchCV
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler 
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn import cluster

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


from sksurv.linear_model import CoxnetSurvivalAnalysis
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.util import Surv

import random as rn
import os
# set random seed
SEED = 0
np.random.seed(SEED)
rn.seed(SEED)
os.environ['PYTHONHASHSEED'] = '0'

In [5]:
# load data
df = pd.read_csv('/omics/odcf/analysis/OE0167_projects/dachs_genetic_data_platform/methylation_markers_new/processed_new/df.csv')

In [6]:
sum(df['PFS']==1)

920

# Dimensionality reduction 

considering the number of N clusters. Given that we have a total of 920 events, the EVP should ideally >10, so I set the number of clusters to 64 or smaller, (64, 32, 16)

In [7]:
cpgs = df.drop(['id', 'Diagnosis_year', 'Age_diag', 'Sex', 'TNM_adj',
             'chemradther', 'timey', 'death_all', 
               'timey_PFS', 'PFS'], axis=1)

In [10]:
agglo = cluster.FeatureAgglomeration(n_clusters=64)
agglo.fit(cpgs)

cpgs_cluster = agglo.transform(cpgs)

In [13]:
cpgs_cluster = pd.DataFrame(cpgs_cluster)

In [24]:
col_names = [f'Cluster{i+1}' for i in range(len(cpgs_cluster.columns))]

In [25]:
cpgs_cluster.columns = col_names

In [26]:
cpgs_cluster.head()

,Cluster1,Cluster2,Cluster3,Cluster4,Cluster5,Cluster6,Cluster7,Cluster8,Cluster9,Cluster10,...,Cluster55,Cluster56,Cluster57,Cluster58,Cluster59,Cluster60,Cluster61,Cluster62,Cluster63,Cluster64
0,0.435582,0.871887,0.757774,0.534852,0.812927,0.629400,0.618443,0.058899,0.289358,0.194509,...,0.304180,0.566950,0.537833,0.800964,0.571615,0.865886,0.455923,0.363930,0.258272,0.685034
1,0.172049,0.887132,0.762574,0.638195,0.795791,0.535838,0.082894,0.055499,0.369591,0.128340,...,0.065896,0.105293,0.565230,0.881836,0.446898,0.903523,0.868393,0.619049,0.156737,0.130190
2,0.377854,0.877360,0.715149,0.520483,0.789480,0.691780,0.550339,0.285024,0.296491,0.201857,...,0.051067,0.418033,0.686618,0.780975,0.570065,0.756916,0.589009,0.509883,0.280478,0.653530
3,0.372801,0.910207,0.794510,0.696294,0.840206,0.727903,0.560590,0.074000,0.378618,0.195602,...,0.355649,0.342046,0.606003,0.566174,0.559459,0.808886,0.833513,0.440694,0.266195,0.710191
4,0.233448,0.879706,0.670757,0.592686,0.798535,0.548289,0.450136,0.224453,0.263479,0.134814,...,0.084027,0.104078,0.510774,0.763692,0.365048,0.908533,0.796375,0.701781,0.176114,0.554617


In [27]:
## join clinical variables
X_reduced = df[['Age_diag','Sex' , 'TNM_adj']].join(cpgs_cluster)

In [28]:
X_reduced.head()

,Age_diag,Sex,TNM_adj,Cluster1,Cluster2,Cluster3,Cluster4,Cluster5,Cluster6,Cluster7,...,Cluster55,Cluster56,Cluster57,Cluster58,Cluster59,Cluster60,Cluster61,Cluster62,Cluster63,Cluster64
0,41,Male,II,0.435582,0.871887,0.757774,0.534852,0.812927,0.629400,0.618443,...,0.304180,0.566950,0.537833,0.800964,0.571615,0.865886,0.455923,0.363930,0.258272,0.685034
1,53,Male,III,0.172049,0.887132,0.762574,0.638195,0.795791,0.535838,0.082894,...,0.065896,0.105293,0.565230,0.881836,0.446898,0.903523,0.868393,0.619049,0.156737,0.130190
2,87,Male,II,0.377854,0.877360,0.715149,0.520483,0.789480,0.691780,0.550339,...,0.051067,0.418033,0.686618,0.780975,0.570065,0.756916,0.589009,0.509883,0.280478,0.653530
3,64,Male,II,0.372801,0.910207,0.794510,0.696294,0.840206,0.727903,0.560590,...,0.355649,0.342046,0.606003,0.566174,0.559459,0.808886,0.833513,0.440694,0.266195,0.710191
4,67,Male,III,0.233448,0.879706,0.670757,0.592686,0.798535,0.548289,0.450136,...,0.084027,0.104078,0.510774,0.763692,0.365048,0.908533,0.796375,0.701781,0.176114,0.554617


In [18]:
y_structured = Surv.from_arrays(df['PFS'], df['timey_PFS'])

# Data preprocessing

In [97]:
### transformer
## determine preprocessor
# Identify the column types
numerical_columns_selector = selector(dtype_exclude=object)
numerical_columns = numerical_columns_selector(X_reduced)

ordinal_columns = ['TNM_adj']
binary_columns = ['Sex']

# Define the transformers
binary_transformer = OneHotEncoder(drop = 'first') ## must only retain one variable otherwise cause error 
ordinal_transformer = OrdinalEncoder(categories=[['I', 'II', 'III']])
continuous_transformer = StandardScaler()

# Define the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('binary', binary_transformer, binary_columns),
        ('ordinal', ordinal_transformer, ordinal_columns),
        ('continuous', continuous_transformer, numerical_columns)
    ])

In [98]:
## determine the scope of alpha
pf = np.ones(X_reduced.shape[1]) # penalty factor, no shrinkage for clinical variables
pf[:3] = 0


X_processed = preprocessor.fit_transform(X_reduced)
# explore the scope of alpha
model = CoxnetSurvivalAnalysis(penalty_factor=pf)
model.fit(X_processed, y_structured)

coefficients_elastic_net = pd.DataFrame(
    model.coef_, index=X_reduced.columns, columns=np.round(model.alphas_, 5)
)

coefficients_elastic_net = coefficients_elastic_net.transpose()

coefficients_elastic_net.reset_index(inplace=True)
coefficients_elastic_net.rename(columns = {'index': 'alpha'}, inplace = True)

n_features = (coefficients_elastic_net.iloc[:, 1:] !=0).sum(axis = 1)

n_features = pd.DataFrame({
 'alpha': coefficients_elastic_net['alpha'],
  'n_features': n_features    
})

CoxnetSurvivalAnalysis(penalty_factor=array([0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]))

In [40]:
n_features

,alpha,n_features
0,0.11162,3
1,0.10171,6
2,0.09267,7
3,0.08444,11
4,0.07694,12
5,0.07010,12
6,0.06388,13
7,0.05820,15
8,0.05303,19
9,0.04832,20


In [99]:
### run the nested CV and tune the hyperparameters
# determine search scope

## l1 ratio, could be fixed to 0.5
l1_scope = []
l1_current = 0.1
while l1_current <= 0.9:
  l1_scope.append(l1_current)
  l1_current +=0.05      

space = {
 "estimator__alphas": [[v] for v in coefficients_elastic_net.alpha],     
 "estimator__l1_ratio": l1_scope        
}

In [100]:
x = X_reduced.to_records(index=False)

# Train the Elastic Net model using nested cross validation

In [108]:
n_folds_outer = 5
n_folds_inner = 3
cv_outer = KFold(n_splits=n_folds_outer, shuffle=True, random_state=SEED)
c_index_censored_scores= []
c_index_ipcw_scores = []
mean_dynamic_AUC = []
integrated_brier_scores = []
best_hyperparameters = []
i = 1
for train_index, test_index in cv_outer.split(x):
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y_structured[train_index], y_structured[test_index]
    
    # preprocessing X variables
    X_scaler = preprocessor.fit(pd.DataFrame(X_train))
    X_train =  X_scaler.transform(pd.DataFrame(X_train))
    X_test = X_scaler.transform(pd.DataFrame(X_test))
    
    lower, upper = np.percentile(y_test["time"], [10, 90])
    model_times = np.arange(lower, upper + 1)
      
    # configure inner CV for tunning
    cv_inner = KFold(n_splits = n_folds_inner, shuffle = True, random_state = SEED)
     
    # define the model
    model =  CoxnetSurvivalAnalysis(penalty_factor = pf, verbose = True, fit_baseline_model=True)

    # define tuning and search
    search = GridSearchCV(as_integrated_brier_score_scorer(model, times=model_times),
                          param_grid=space, cv=cv_inner, n_jobs = -1)
      
    result = search.fit(X_train, y_train)  
  
   ## store the best hyperparameters
    best_hyperparameters.append(result.best_params_)
   
   ## store the selected features and coefficients   
    best_model = result.best_estimator_.estimator_ 
    df_coef = pd.DataFrame(best_model.coef_)
    df_coef['feature'] = X_reduced.columns
    df_coef = df_coef[df_coef.iloc[:, 0] !=0]
    df_coef.to_csv(f'/omics/odcf/analysis/OE0167_projects/dachs_genetic_data_platform/methylation_markers_new/temp_results/df_coef_cvc64{i}.csv', index=False)  
    i += 1
      
   # evaluate the best model on the hold out dataset
    yhat = best_model.predict(X_test)   
   
    c_index_censored = concordance_index_censored(y_test['event'], y_test['time'], yhat)[0]
    c_index_censored_scores.append(c_index_censored)

    c_index_ipcw = concordance_index_ipcw(y_train, y_test, yhat)[0]
    c_index_ipcw_scores.append(c_index_ipcw)

    dauc = cumulative_dynamic_auc(y_train, y_test, yhat, model_times)
    mean_dynamic_AUC.append(dauc[-1])

    survs = best_model.predict_survival_function(X_test)
    preds = np.asarray([[fn(t) for t in model_times] for fn in survs])  
    brier_score_val = integrated_brier_score(y_train, y_test, preds, model_times)
    integrated_brier_scores.append(brier_score_val)   

max update after 3 iterations increased from 0.000677387 to 0.000802011
max update after 3 iterations increased from 0.000695632 to 0.000836625
max update after 3 iterations increased from 0.00067102 to 0.000812067
max update after 3 iterations increased from 0.000663865 to 0.000802524
max update after 3 iterations increased from 0.000652367 to 0.000805734
max update after 3 iterations increased from 0.000652673 to 0.000822599
max update after 3 iterations increased from 0.000649018 to 0.000842417
max update after 3 iterations increased from 0.000664291 to 0.00087947
max update after 3 iterations increased from 0.000649996 to 0.000879436
max update after 3 iterations increased from 0.000725342 to 0.000831603
max update after 3 iterations increased from 0.000642979 to 0.000847274
max update after 3 iterations increased from 0.000741308 to 0.000848496
max update after 3 iterations increased from 0.000636308 to 0.000854176
max update after 3 iterations increased from 0.000696867 to 0.0008

In [102]:
## bind the best hyperparameters and performance 
folds = list(range(1, n_folds_outer+1))
performance_nestedcv = pd.DataFrame({
      'Folds' : folds,
       'Hyperparameters': best_hyperparameters,
       'C_index_censored': c_index_censored_scores,
      'C_index_ipcw': c_index_ipcw_scores,
       'Mean dynamic AUC': mean_dynamic_AUC,
      'Integrated Brier score': integrated_brier_scores})

col_names = performance_nestedcv.columns
summary = pd.Series({
col_names[0]: 'mean',
 col_names[1]: '-', 
 col_names[2]: performance_nestedcv.iloc[:, 2].mean(),
 col_names[3]: performance_nestedcv.iloc[:, 3].mean(),
 col_names[4]: performance_nestedcv.iloc[:, 4].mean(),
 col_names[5]: performance_nestedcv.iloc[:, 5].mean(),
})
   

In [103]:
performance_nestedcv = performance_nestedcv.append(summary, ignore_index=True)
performance_nestedcv

/tmp/ipykernel_18316/2078093159.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  performance_nestedcv = performance_nestedcv.append(summary, ignore_index=True)


,Folds,Hyperparameters,C_index_censored,C_index_ipcw,Mean dynamic AUC,Integrated Brier score
0,1,"{'estimator__alphas': [0.06388], 'estimator__l...",0.640666,0.654141,0.675123,0.201691
1,2,"{'estimator__alphas': [0.11162], 'estimator__l...",0.620147,0.644355,0.626309,0.197374
2,3,"{'estimator__alphas': [0.11162], 'estimator__l...",0.690137,0.672359,0.720614,0.180950
3,4,"{'estimator__alphas': [0.04012], 'estimator__l...",0.665624,0.676850,0.692265,0.185460
4,5,"{'estimator__alphas': [0.10171], 'estimator__l...",0.628081,0.647541,0.647661,0.201313
5,mean,-,0.648931,0.659049,0.672394,0.193358


In [104]:
new_names = ['features_df1', 'features_df2', 'features_df3', 'features_df4', 'features_df5']
j = 1
for i in range(0,5):
      j = i + 1
      new_names[i]= pd.read_csv(f'/omics/odcf/analysis/OE0167_projects/dachs_genetic_data_platform/methylation_markers_new/temp_results/df_coef_cvc64{j}.csv')
coef_list = [
      new_names[0]['feature'],
      new_names[1]['feature'],
      new_names[2]['feature'],
      new_names[3]['feature'],
      new_names[4]['feature']
]

def find_intersection(series1, series2):
    return pd.Series(list(set(series1) & set(series2)))

intersection = reduce(find_intersection, coef_list)

intersection ## still only four genes wer

0    Age_diag
1         Sex
2     TNM_adj
dtype: object

In [65]:
### refit the cox model
X = X_reduced[intersection]
Y = df[['timey_PFS', 'PFS']]

x = X.to_records(index=False)
y_structured = Surv.from_arrays(Y['PFS'], Y['timey_PFS'])

In [66]:
# Identify the column types
numerical_columns_selector = selector(dtype_exclude=object)
numerical_columns = numerical_columns_selector(X)

ordinal_columns = ['TNM_adj']
binary_columns = ['Sex']

# Define the transformers
binary_transformer = OneHotEncoder(drop = 'first') ## must only retain one variable otherwise cause error 
ordinal_transformer = OrdinalEncoder(categories=[['I', 'II', 'III']])
continuous_transformer = StandardScaler()

# Define the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('binary', binary_transformer, binary_columns),
        ('ordinal', ordinal_transformer, ordinal_columns),
        ('continuous', continuous_transformer, numerical_columns)
    ])

In [67]:
c_index_censored_scores= []
c_index_ipcw_scores = []
mean_dynamic_AUC = []
brier_scores = []
n_folds = 5
# Create a StratifiedKFold object
kf = KFold(n_splits=n_folds, shuffle=True, random_state=SEED)
for train_index, test_index in kf.split(x):
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y_structured[train_index], y_structured[test_index]
    
    # preprocessing X variables
    X_scaler = preprocessor.fit(pd.DataFrame(X_train))
    X_train =  X_scaler.transform(pd.DataFrame(X_train))
    X_test = X_scaler.transform(pd.DataFrame(X_test))  

    # fit the model
    model = CoxPHSurvivalAnalysis()
    model.fit(X_train, y_train)  
    
    # Compute censored C-index 
    c_index_censored = concordance_index_censored(y_test['event'], y_test['time'], model.predict(X_test))[0]
    #print(f"C-index_censored: {c_index_censored}")
    c_index_censored_scores.append(c_index_censored)
      
    # Compute IPCW C-index 
    c_index_ipcw = concordance_index_ipcw(y_train, y_test, model.predict(X_test))[0]
    #print(f"C-index_IPCW: {c_index_ipcw}")
    c_index_ipcw_scores.append(c_index_ipcw)  
      
    # compute cumulative AUC
    lower, upper = np.percentile(y_test["time"], [10, 90])
    times = np.arange(lower, upper + 1)
      
    dauc = cumulative_dynamic_auc(y_train, y_test, model.predict(X_test), times)
    mean_dynamic_AUC.append(dauc[-1])
      
    # Compute Brier score
    survs = model.predict_survival_function(X_test)
    preds = np.asarray([[fn(t) for t in times] for fn in survs])  
    brier_score_val = integrated_brier_score(y_train, y_test, preds, times)
    #print(f"Brier score: {brier_score_val}")
    brier_scores.append(brier_score_val)
             

CoxPHSurvivalAnalysis()

CoxPHSurvivalAnalysis()

CoxPHSurvivalAnalysis()

CoxPHSurvivalAnalysis()

CoxPHSurvivalAnalysis()

In [68]:
folds = list(range(1, n_folds+1))
folds.append('mean')
c_index_censored_scores.append(sum(c_index_censored_scores)/n_folds)
c_index_ipcw_scores.append(sum(c_index_ipcw_scores)/n_folds)
mean_dynamic_AUC.append(sum(mean_dynamic_AUC)/n_folds)
brier_scores.append(sum(brier_scores)/n_folds)

## summarize all metrics into a pandas dataframe
performance_5cv = pd.DataFrame({
      'Folds' : folds,
       'C_index_censored': c_index_censored_scores,
      'C_index_ipcw': c_index_ipcw_scores,
       'Mean dynamic AUC': mean_dynamic_AUC,
      'Integrated Brier score': brier_scores})

performance_5cv.iloc[:, 1:] = performance_5cv.iloc[:, 1:].round(2)
performance_5cv

,Folds,C_index_censored,C_index_ipcw,Mean dynamic AUC,Integrated Brier score
0,1,0.65,0.67,0.68,0.20
1,2,0.62,0.65,0.63,0.20
2,3,0.70,0.67,0.73,0.18
3,4,0.67,0.68,0.71,0.18
4,5,0.63,0.65,0.65,0.20
5,mean,0.66,0.66,0.68,0.19


Refit the Cox model using all the CpGs corresponding to the selected clusters


In [70]:
cpgs_cluster = pd.DataFrame({
 'cpg': cpgs.columns,
  'cluster': agglo.labels_    
})

In [72]:
## select the Cpgs 
cpgs_selected = [18, 24, 32, 43, 8, 58, 41]
cpgs_cluster_filter = cpgs_cluster[cpgs_cluster['cluster'].isin(cpgs_selected)]

In [75]:
cpgs_cluster_filter.shape
cpgs_cluster_filter.head()

(97, 2)

,cpg,cluster
4,cg08159663,8
17,cg09820519,24
18,cg26312951,8
26,cg20971158,8
32,cg16096311,24


In [77]:
select = cpgs_cluster_filter['cpg']

In [89]:
select = select.append(pd.Series(['Sex', 'TNM_adj', 'Age_diag']))

/tmp/ipykernel_18316/3622835420.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  select = select.append(pd.Series(['Sex', 'TNM_adj', 'Age_diag']))


In [90]:
### refit the cox model
X = df[select]
Y = df[['timey_PFS', 'PFS']]

x = X.to_records(index=False)
y_structured = Surv.from_arrays(Y['PFS'], Y['timey_PFS'])

In [93]:
# Identify the column types
numerical_columns_selector = selector(dtype_exclude=object)
numerical_columns = numerical_columns_selector(X)

ordinal_columns = ['TNM_adj']
binary_columns = ['Sex']

# Define the transformers
binary_transformer = OneHotEncoder(drop = 'first') ## must only retain one variable otherwise cause error 
ordinal_transformer = OrdinalEncoder(categories=[['I', 'II', 'III']])
continuous_transformer = StandardScaler()

# Define the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('binary', binary_transformer, binary_columns),
        ('ordinal', ordinal_transformer, ordinal_columns),
        ('continuous', continuous_transformer, numerical_columns)
    ])

In [95]:
c_index_censored_scores= []
c_index_ipcw_scores = []
mean_dynamic_AUC = []
brier_scores = []
n_folds = 5
# Create a StratifiedKFold object
kf = KFold(n_splits=n_folds, shuffle=True, random_state=SEED)
for train_index, test_index in kf.split(x):
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y_structured[train_index], y_structured[test_index]
    
    # preprocessing X variables
    X_scaler = preprocessor.fit(pd.DataFrame(X_train))
    X_train =  X_scaler.transform(pd.DataFrame(X_train))
    X_test = X_scaler.transform(pd.DataFrame(X_test))  

    # fit the model
    model = CoxPHSurvivalAnalysis()
    model.fit(X_train, y_train)  
    
    # Compute censored C-index 
    c_index_censored = concordance_index_censored(y_test['event'], y_test['time'], model.predict(X_test))[0]
    #print(f"C-index_censored: {c_index_censored}")
    c_index_censored_scores.append(c_index_censored)
      
    # Compute IPCW C-index 
    c_index_ipcw = concordance_index_ipcw(y_train, y_test, model.predict(X_test))[0]
    #print(f"C-index_IPCW: {c_index_ipcw}")
    c_index_ipcw_scores.append(c_index_ipcw)  
      
    # compute cumulative AUC
    lower, upper = np.percentile(y_test["time"], [10, 90])
    times = np.arange(lower, upper + 1)
      
    dauc = cumulative_dynamic_auc(y_train, y_test, model.predict(X_test), times)
    mean_dynamic_AUC.append(dauc[-1])
      
    # Compute Brier score
    survs = model.predict_survival_function(X_test)
    preds = np.asarray([[fn(t) for t in times] for fn in survs])  
    brier_score_val = integrated_brier_score(y_train, y_test, preds, times)
    #print(f"Brier score: {brier_score_val}")
    brier_scores.append(brier_score_val)

CoxPHSurvivalAnalysis()

CoxPHSurvivalAnalysis()

CoxPHSurvivalAnalysis()

CoxPHSurvivalAnalysis()

CoxPHSurvivalAnalysis()

In [96]:
folds = list(range(1, n_folds+1))
folds.append('mean')
c_index_censored_scores.append(sum(c_index_censored_scores)/n_folds)
c_index_ipcw_scores.append(sum(c_index_ipcw_scores)/n_folds)
mean_dynamic_AUC.append(sum(mean_dynamic_AUC)/n_folds)
brier_scores.append(sum(brier_scores)/n_folds)

## summarize all metrics into a pandas dataframe
performance_5cv = pd.DataFrame({
      'Folds' : folds,
       'C_index_censored': c_index_censored_scores,
      'C_index_ipcw': c_index_ipcw_scores,
       'Mean dynamic AUC': mean_dynamic_AUC,
      'Integrated Brier score': brier_scores})

performance_5cv.iloc[:, 1:] = performance_5cv.iloc[:, 1:].round(2)
performance_5cv

,Folds,C_index_censored,C_index_ipcw,Mean dynamic AUC,Integrated Brier score
0,1,0.65,0.66,0.67,0.21
1,2,0.61,0.62,0.62,0.21
2,3,0.66,0.64,0.70,0.19
3,4,0.65,0.66,0.68,0.19
4,5,0.61,0.62,0.63,0.21
5,mean,0.64,0.64,0.66,0.20
